<h2 style="text-align: center; font-weight: bold">Genetic Algorithm Crossovers for Travelling Salesperson Problem</h2>

In [40]:
import random

In [41]:
def tsp_printer(cities: list[str], list_name: str, strong_genes_seq: tuple[int, int], color_code: int) -> None:
  print(list_name, end = ": ")
  for i in range(25):
    if i not in range(strong_genes_seq[0], strong_genes_seq[1]):
      print(cities[i], end = " ")
    else:
      print(f"\033[38;5;{color_code}m{cities[i]}", end = " \033[0m")
  print()

In [42]:
# P_n - parent_, C_n - children
P1 = [chr(i) for i in range(65, 91)]
P2 = [chr(i) for i in range(65, 91)]
random.shuffle(P1) 
random.shuffle(P2)
tsp_printer(P1, "P1", (9, 16), 208)
tsp_printer(P2, "P2", (9, 16), 208)

P1: D U T E O G M S V I B H X P R F L Y C A J K N Z Q 
P2: K Z M N X J A F Y G W U O C E D S B T V I H L P R 


#### 1. Ordered Crossover

In [43]:
def ordered_crossover(P1: list[str], P2: list[str], strong_genes_seq: tuple[int, int]) -> tuple[list[str], list[str]]:  
  """
  "strong_genes_seq" should have the start and end indices of strong genes { [start_index, end_index) },\n 
  i.e, the range of genes that are copied untouched in corresponding children (P_n Ξ C_n)
  """
  C1, C2 = [], []
  for i in range(strong_genes_seq[0], strong_genes_seq[1]):
    C1.append(P1[i])
    C2.append(P2[i])

  weak_genes_C1 = list(filter(lambda x: x if x not in C1 else None, P2))   # 'x' is Truthy when parameter is 'x' whereas 'None' is falsy when param is 'x'... that's why not using explicit boolean returns is still working
  weak_genes_C2 = list(filter(lambda x: x if x not in C2 else None, P1))
  
  C1 = weak_genes_C1[:9] + C1 + weak_genes_C1[10:]
  C2 = weak_genes_C2[:9] + C2 + weak_genes_C2[10:]

  return C1, C2

# not my first time using the filter function but this is a good read: https://realpython.com/python-filter-function/

In [44]:
C1, C2 = ordered_crossover(P1, P2, (9, 16))
print("Parents were: ")
tsp_printer(P1, "P1", (9, 16), 208)
tsp_printer(P2, "P2", (9, 16), 208)

print("\nChildren are: ")
tsp_printer(C1, "C1", (9, 16), 51)
tsp_printer(C2, "C2", (9, 16), 51)

Parents were: 
P1: D U T E O G M S V I B H X P R F L Y C A J K N Z Q 
P2: K Z M N X J A F Y G W U O C E D S B T V I H L P R 

Children are: 
C1: K Z M N J A Y G W I B H X P R F O C E D S T V L Q 
C2: T M S V I B H X P G W U O C E D F L Y A J K N Z Q 


#### 2. Partially Mapped Crossover

In [45]:
def partially_mapped_crossover(P1: list[str], P2: list[str]) -> tuple[list[str], list[str]]:
  C1, C2 = [], []
  
  return C1, C2